In [1]:
#!pip install pymongo

In [2]:
import pymongo as mg
import pandas as pd 

In [3]:
client = mg.MongoClient(host='mongodb://localhost:27017') # MongoDB 서버에 연결하는 주소 설정

In [4]:
database = client['db_medicals'] #Database 선택

In [5]:
collection = database['RecurrenceOfSurgery'] # collection 선택

In [6]:
cursor = collection.find({})
data_list = list(cursor)

In [7]:
df_ROS = pd.DataFrame(data_list)
df_ROS.head(2)

,_id,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,6545ec9ae338bba9485736de,0,1PT,22.8,3,51.0,0.0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,6545ec9ae338bba9485736df,1,2PT,44.9,4,26.0,0.0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


In [8]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     1894 non-null   object 
 1   Unnamed: 0              1894 non-null   int64  
 2   환자ID                    1894 non-null   object 
 3   Large Lymphocyte        1894 non-null   float64
 4   Location of herniation  1894 non-null   int64  
 5   ODI                     462 non-null    float64
 6   가족력                     1843 non-null   float64
 7   간질성폐질환                  1894 non-null   int64  
 8   고혈압여부                   1894 non-null   int64  
 9   과거수술횟수                  1894 non-null   int64  
 10  당뇨여부                    1894 non-null   int64  
 11  말초동맥질환여부                1894 non-null   int64  
 12  빈혈여부                    1894 non-null   int64  
 13  성별                      1894 non-null   int64  
 14  스테로이드치료                 1894 non-null   

### 분석기준 : 연령대에 따라 발생되는 질병의 종류 분포

##### 불필요한 columns drop

In [9]:
df_ROS.columns  

Index(['_id', 'Unnamed: 0', '환자ID', 'Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수',
       '당뇨여부', '말초동맥질환여부', '빈혈여부', '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환',
       '암발병여부', '연령', '우울증여부', '입원기간', '입원일자', '종양진행여부', '직업', '체중', '퇴원일자',
       '헤모글로빈수치', '혈전합병증여부', '환자통증정도', '흡연여부', '통증기간(월)', '수술기법', '수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도',
       'Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)',
       'Vaccum disc', '골밀도', '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

In [10]:
df_ROS_drops = df_ROS.drop(columns=['_id','Unnamed: 0','Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력','간질성폐질환','과거수술횟수','성별', '스테로이드치료','신장','입원기간', '입원일자', '직업', '체중', '퇴원일자',
       '헤모글로빈수치','환자통증정도', '흡연여부', '통증기간(월)', '수술기법','수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도','Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)',
       'Vaccum disc', '골밀도', '디스크단면적', '디스크위치', '척추이동척도' ])
df_ROS_drops.head(2)

,환자ID,고혈압여부,당뇨여부,말초동맥질환여부,빈혈여부,신부전여부,심혈관질환,암발병여부,연령,우울증여부,종양진행여부,혈전합병증여부,척추전방위증
0,1PT,0,0,0,0,0,0,0,66,0,0,0,0
1,2PT,0,0,0,0,0,0,0,47,0,0,0,0


##### 데이터 타입 확인

In [11]:
df_ROS_drops.describe() # 수치형

,고혈압여부,당뇨여부,말초동맥질환여부,빈혈여부,신부전여부,심혈관질환,암발병여부,연령,우울증여부,종양진행여부,혈전합병증여부,척추전방위증
count,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000,1894.000000
mean,0.130940,0.062830,0.004224,0.024287,0.025343,0.035903,0.013728,41.463569,0.023231,0.003168,0.002112,0.032207
std,0.337424,0.242721,0.064871,0.153980,0.157207,0.186097,0.116388,13.444629,0.154143,0.056210,0.045919,0.176596
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,15.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,32.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,41.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,49.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,86.000000,2.000000,1.000000,1.000000,1.000000


In [12]:
df_ROS_drops.describe(include=object) # 범주형

,환자ID
count,1894
unique,1894
top,1PT
freq,1


In [13]:
df_ROS_drops['당뇨여부'].value_counts()

당뇨여부
0    1775
1     119
Name: count, dtype: int64

In [14]:
df_ROS_drops['당뇨여부'].unique()

array([0, 1], dtype=int64)